In [2]:
from scipy.spatial import cKDTree
import numpy as np
import pickle


In [10]:
def getEigenN(matrix):
    cov = np.cov(matrix.T)
    val, vec = np.linalg.eig(cov)
    srt = val.argsort()[::-1]
    val = val[srt]
    vec = vec[:,srt]
    if (val[2] < 0):
        val[2] = 0
    val = val/sum(val)
    return val, vec

In [11]:
def getHighIntensityRoadPoints(data, threshold, clf):
    XYZ = data[:,0:3]
    XYZKD = cKDTree(XYZ, leafsize=30)
    features = []
    roadIndex = []
    for i in range(data.shape[0]):
        #intensity index
        if (data[i][3] >= 190):
            neighborIndex = XYZKD.query_ball_point(XYZ[i], 1)
            if len(neighborIndex) >= 3:
                neighbors = XYZ[neighborIndex]
                val, vec = getEigenN(neighbors)
                vec = vec.T
                EV1, EV2, EV3 = val[0],val[1],val[2]
                VT1, VT2, VT3 = vec[0],vec[1],vec[2]
        
                for j in range(3):
                    VT3[j] = abs(VT3[j])
                vertical = -(VT3[2]/sum(VT3) - 1)
                
                feature = [(EV1-EV2)/EV1, (EV2-EV3)/EV1, EV3/EV1, vertical]
                features.append(feature)
                roadIndex.append(i)
    results = clf.predict(features)
    ##

In [5]:
XYZI = np.genfromtxt("withGround01.csv", delimiter=",", usecols=(0,1,2,3))
with open('roadClassifier.pickle', 'rb') as f:
    clf = pickle.load(f)


In [ ]:
highIntensityRoadPoints = getHighIntensityRoadPoints(XYZI, 190, clf)

In [6]:
XYZ = XYZI[:,0:3]

In [9]:
XYZI[3][3]

94.0